In [1]:
from cobra import Model
from cobra.io import read_sbml_model

In [2]:
model = read_sbml_model('model_lipids.xml')

3046 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


In [ ]:
from cobra import Model
model_cyano = Model()

In [32]:
id_metabolites = [metabolite.id for metabolite in model.metabolites] 
metabolites_to_add = []
reactions_to_add = []


for metabolite in model.metabolites:
    compartment = metabolite.id.split("__")[-1] 
    newloc = metabolite.id.replace(compartment,"cytop") 
    if newloc not in id_metabolites:
        metabolite.id = newloc
        id_metabolites.append(newloc)
    if metabolite.id.endswith("__cytop"):
        metabolites_to_add.append(metabolite)


id_reactions = [reaction.id for reaction in model.reactions]

for reaction in model.reactions:
    reactions = reaction.id.split("__")[-1]
    newloc1 = reaction.id.replace(reactions, "cytop")
    if newloc1 not in id_reactions:
        reaction.id = newloc1
        id_reactions.append(newloc1) 
    if reaction.id.endswith("__cytop"):
        for metabolite in reaction.metabolites:
            if not metabolite.id.endswith("__cytop"):
                st = reaction.metabolites[metabolite]
                reaction.add_metabolites({metabolite: -st, model.metabolites.get_by_id(metabolite.id.split("__")[0] + "__cytop"): st})
        reactions_to_add.append(reaction)

In [33]:
model_cyano.add_metabolites(metabolites_to_add)

In [34]:
model_cyano.add_reactions(reactions_to_add)

In [37]:
groups_to_add = []
for group in model.groups:
    if any([reaction in reactions_to_add for reaction in group.members]):
        to_remove = []
        for reaction in group.members:
            if reaction not in reactions_to_add:
                to_remove.append(reaction)
        group.remove_members(to_remove)
        groups_to_add.append(group)
model_cyano.add_groups(groups_to_add)

In [38]:
from cobra.io import write_sbml_model

write_sbml_model(model_cyano, "model_lipids_cyano.xml")

In [14]:
def get_reactions(id_lipid, reactions_list):
    lipid = model_cyano.metabolites.get_by_id(id_lipid + "__cytop")
    for reaction in lipid.reactions:
        if lipid in reaction.products or reaction.lower_bound < 0:
            reactions_list.add(reaction)
            for reactant in reaction.reactants:
                reactions_to_add = get_reactions(reactant.id.split("__")[0], reactions_list)
                reactions_list.add([e for e in reactions_to_add])
    return reactions_list

In [1]:
from cobra.io import read_sbml_model
model_cyano = read_sbml_model('model_lipids_cyano.xml')

'' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


In [2]:
id_lipid = ["mgdg140183", "mgdg160184", "mgdg160183", "mgdg160182", "mgdg160181",
"mgdg182183",
"mgdg181183",
"mgdg160200",
"mgdg190210"]

id_metabolites = [metabolite.id for metabolite in model_cyano.metabolites]

In [3]:
for lipid in id_lipid:
    if lipid + "__cytop" in id_metabolites:
        reactions_list = get_reactions(lipid, set())


NameError: name 'get_reactions' is not defined